# Financial Analysis Application

In this notebook, you will create agents to review financial documents and generate a Market Price Analysis (MPA) using the CrewAI framewkor.


In [2]:
# Installation
%pip install -r requirements.txt
%pip install ipywidgets
%jupyter nbextension enable --py widgetsnbextension

In [ ]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
import os
from utils import get_openai_api_key
import ipywidgets as widgets
from IPython.display import display, Markdown
import pandas as pd
from io import BytesIO

# Set the OpenAI API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OpenAI API key not found. Please set it in your .env file or environment variables.")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

In [ ]:
# Create the file upload widget
upload = widgets.FileUpload(
    accept='application/pdf, text/csv, application/vnd.ms-excel, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',  # Specify accepted file types
    multiple=True  # Allow multiple file uploads
)

# Display the widget
display(upload)

# Function to process uploaded files
def handle_upload(change):
    files_content = {}
    for filename, file_info in upload.value.items():
        if filename.endswith('.csv'):
            df = pd.read_csv(BytesIO(file_info['content']))
            files_content[filename] = df
            display(df.head())
        elif filename.endswith('.xlsx'):
            df = pd.read_excel(BytesIO(file_info['content']))
            files_content[filename] = df
            display(df.head())
        elif filename.endswith('.pdf'):
            with open(filename, 'wb') as f:
                f.write(file_info['content'])
            files_content[filename] = file_info['content']
            display(f"PDF file uploaded: {filename}")
    return files_content

# Attach the handler to the file upload widget
upload.observe(handle_upload, names='value')

In [ ]:
# Creating Agents

# Agent: Document Reviewer
class DocumentReviewerAgent(Agent):
    def execute(self, inputs):
        file_content = inputs["file_content"]
        # Process the DataFrame and extract key financial figures
        financial_figures = {
            "Gross Revenue": file_content["Gross Revenue"].sum(),
            "Cost of Goods Sold (COGS)": file_content["COGS"].sum(),
            # Add other key figures as required
        }
        addback_figures = {
            "Meals and Entertainment": file_content["Meals and Entertainment"].sum(),
            "Vehicles": file_content["Vehicles"].sum(),
            # Add other addback figures as required
        }
        return financial_figures, addback_figures

doc_reviewer = DocumentReviewerAgent(
    role="Document Reviewer",
    goal="Review financial statements and extract key financial figures. Your work is the basis for the Financial Analyst and Addback Analyst to complete their respective financial reports.",
    backstory="You're tasked with reviewing financial statements provided by a potential seller. Your goal is to extract and provide key financial figures needed for further analysis for the Financial and Addback Analyst. Key Figures include Gross Revenue, Cost of Goods Sold (COGS), Gross Profit, Operating Expense, Net Income, Depreciation, Income Tax, Amortization, Interest on loans/financing, Owners health insurance, Owners retirement contributions, Salary for non working family member, Owners Salary, Owner's Estimated Payroll Taxes, Total Bank Financing Addbacks, and Banker's Adjusted Net Income for the financial analyst, and then provide the following key financials to the Addback Analyst. Include any Meals and Entertainment, Vehicles, Assets, Real Estate, Travel Expenses, Fines and Penalties, Life Insurance Premiums for Key Employees, Interest on Loans for Tax-Exempt Investments, Depreciation on Personal Assets, Cost of Goods Sold (COGS) for Non-Business Products or Services, Losses from Hobbies or Non-Business Activities, Expenses Paid with Personal Funds, Owner’s Compensation, Non-Deductible Taxes, Business Gifts, Depreciation and Amortization Adjustments, Charitable Contributions of Non-Investment Property, Political Contributions, Federal Estate and Gift Taxes, Depreciation on Listed Property, Losses from Sale or Exchange of Listed Property, Depreciation on a Home Office, Unreimbursed Employee Business Expenses, Tax Penalties and Fines, Lobbying Expenses, Club Dues, Mileage Deductions Documentation Requirements, Bank Fines and Fees, Credit Card Fees, and Deductibility of Life Insurance Premiums.",
    allow_delegation=False,
    verbose=True
)

# Agent: Financial Analyst
class FinancialAnalystAgent(Agent):
    def execute(self, inputs):
        financial_figures = inputs["financial_figures"]
        # Analyze the financial figures
        # Ensure all details required for the MPA are available
        analysis_report = {
            "Revenue Analysis": f"Gross Revenue: {financial_figures['Gross Revenue']}",
            "COGS Analysis": f"Cost of Goods Sold: {financial_figures['Cost of Goods Sold (COGS)']}",
            # Add other analysis details as required
        }
        return analysis_report

financial_analyst = FinancialAnalystAgent(
    role="Financial Analyst",
    goal="Analyze the extracted financial figures and provide detailed insights.",
    backstory="You're tasked with analyzing the financial figures extracted by the Document Reviewer. Your goal is to ensure all of the details required for the MPA are available, and available for the MPA Preparer. If you do not receive everything, have the Document Reviewer review the document again, and provide any figures that would allow you to calculate or deduce the figures you’re missing.",
    allow_delegation=False,
    verbose=True
)

# Agent: Addback Analyst
class AddbackAnalystAgent(Agent):
    def execute(self, inputs):
        addback_figures = inputs["addback_figures"]
        # Analyze the addback figures
        # Ensure all details required for the MPA are available
        addback_report = {
            "Meals and Entertainment Analysis": f"Meals and Entertainment: {addback_figures['Meals and Entertainment']}",
            "Vehicles Analysis": f"Vehicles: {addback_figures['Vehicles']}",
            # Add other addback analysis details as required
        }
        return addback_report

addback_analyst = AddbackAnalystAgent(
    role="Addback Analyst",
    goal="Analyze the granular numbers associated with addbacks and provide details.",
    backstory="You're tasked with analyzing the granular numbers associated with addbacks provided by the Document Reviewer. Your goal is to ensure all of the details required for the MPA are available, and available for the MPA Preparer. If you do not receive everything, have the Document Reviewer review the document again, and provide any figures that would allow you to calculate or deduce the figures you’re missing.",
    allow_delegation=False,
    verbose=True
)

# Agent: MPA Preparer
class MPAPreparerAgent(Agent):
    def execute(self, inputs):
        financial_analysis_report = inputs["financial_analysis_report"]
        addback_analysis_report = inputs["addback_analysis_report"]
        # Compile the insights from the financial and addback analyses
        mpa_report = {
            "Financial Analysis": financial_analysis_report,
            "Addback Analysis": addback_analysis_report,
            "Questions": ["List any questions where the data is questionable."]
        }
        return mpa_report

mpa_preparer = MPAPreparerAgent(
    role="MPA Preparer",
    goal="Prepare a Market Price Analysis (MPA) and list questions for the seller.",
    backstory="You're tasked with preparing the Market Price Analysis (MPA) using the data provided by the Financial Analyst and Addback Analyst. Your goal is to complete the MPA Valuation as fully as possible, and then compile a list of questions where the data is questionable.",
    allow_delegation=False,
    verbose=True
)

# Agent: Editor
class EditorAgent(Agent):
    def execute(self, inputs):
        mpa_report = inputs["mpa_report"]
        # Review the MPA and ensure accuracy and clarity
        final_mpa_report = f"Final MPA Report:\n{mpa_report}"
        return final_mpa_report

editor = EditorAgent(
    role="Editor",
    goal="Review the MPA and questions to ensure accuracy and clarity.",
    backstory="You're tasked with reviewing the Market Price Analysis (MPA) and the list of questions generated by the MPA Preparer. Your goal is to ensure accuracy and clarity in the final documents.",
    allow_delegation=False,
    verbose=True
)

In [ ]:
# Creating Tasks

# Task: Review Documents
review_docs = Task(
    description=(
        "1. Review the financial statements provided by the seller.\n"
        "2. Extract key financial figures such as Gross Revenue, Cost of Goods Sold (COGS), Gross Profit, Operating Expense, Net Income, Depreciation, Income Tax, Amortization, Interest on loans/financing, Owners health insurance, Owners retirement contributions, Salary for non working family member, Owners Salary, Owner's Estimated Payroll Taxes, Total Bank Financing Addbacks, and Banker's Adjusted Net Income for the financial analyst.\n"
        "3. Provide the following key financials to the Addback Analyst: Meals and Entertainment, Vehicles, Assets, Real Estate, Travel Expenses, Fines and Penalties, Life Insurance Premiums for Key Employees, Interest on Loans for Tax-Exempt Investments, Depreciation on Personal Assets, Cost of Goods Sold (COGS) for Non-Business Products or Services, Losses from Hobbies or Non-Business Activities, Expenses Paid with Personal Funds, Owner’s Compensation, Non-Deductible Taxes, Business Gifts, Depreciation and Amortization Adjustments, Charitable Contributions of Non-Investment Property, Political Contributions, Federal Estate and Gift Taxes, Depreciation on Listed Property, Losses from Sale or Exchange of Listed Property, Depreciation on a Home Office, Unreimbursed Employee Business Expenses, Tax Penalties and Fines, Lobbying Expenses, Club Dues, Mileage Deductions Documentation Requirements, Bank Fines and Fees, Credit Card Fees, and Deductibility of Life Insurance Premiums."
    ),
    expected_output="Structured financial figures extracted from the documents for the Financial and Addback Analysts.",
    agent=doc_reviewer,
)

# Task: Analyze Financial Figures
analyze_financials = Task(
    description=(
        "1. Analyze the extracted financial figures provided by the Document Reviewer.\n"
        "2. Ensure all details required for the MPA are available.\n"
        "3. If any figures are missing, instruct the Document Reviewer to review the documents again and provide the necessary figures."
    ),
    expected_output="Detailed financial analysis report, ensuring completeness of data for MPA preparation.",
    agent=financial_analyst,
)

# Task: Analyze Addbacks
analyze_addbacks = Task(
    description=(
        "1. Analyze the granular numbers associated with addbacks provided by the Document Reviewer.\n"
        "2. Ensure all details required for the MPA are available.\n"
        "3. If any figures are missing, instruct the Document Reviewer to review the documents again and provide the necessary figures."
    ),
    expected_output="Detailed addback analysis report, ensuring completeness of data for MPA preparation.",
    agent=addback_analyst,
)

# Task: Prepare MPA
prepare_mpa = Task(
    description=(
        "1. Compile the insights from the financial and addback analyses.\n"
        "2. Prepare a comprehensive Market Price Analysis (MPA).\n"
        "3. Generate a list of questions where the data is questionable."
    ),
    expected_output="Comprehensive MPA report with a list of questions.",
    agent=mpa_preparer,
)

# Task: Review MPA
review_mpa = Task(
    description=(
        "1. Review the MPA and the list of questions generated by the MPA Preparer.\n"
        "2. Ensure accuracy and clarity in the final documents."
    ),
    expected_output="Final reviewed MPA report and list of questions.",
    agent=editor,
)

In [ ]:
# Creating the Crew

crew = Crew(
    agents=[doc_reviewer, financial_analyst, addback_analyst, mpa_preparer, editor],
    tasks=[review_docs, analyze_financials, analyze_addbacks, prepare_mpa, review_mpa],
    verbose=2
)

In [ ]:
# Running the Crew with Uploaded Files

# Define a function to run the crew with the uploaded files
def run_crew_with_files(files_content):
    for filename, content in files_content.items():
        if isinstance(content, pd.DataFrame):
            print(f"Processing {filename}...")
            # Extract key financial figures using the Document Reviewer agent
            financial_figures, addback_figures = doc_reviewer.execute({"file_content": content})
            
            # Ensure the financial figures are passed to the Financial Analyst and Addback Analyst
            financial_analysis_report = financial_analyst.execute({"financial_figures": financial_figures})
            addback_analysis_report = addback_analyst.execute({"addback_figures": addback_figures})
            
            # Compile the insights from both analysts
            mpa_input = {
                "financial_analysis_report": financial_analysis_report,
                "addback_analysis_report": addback_analysis_report
            }
            
            # Prepare the MPA using the MPA Preparer agent
            mpa_report = mpa_preparer.execute(mpa_input)
            
            # Review the MPA using the Editor agent
            final_mpa_report = editor.execute({"mpa_report": mpa_report})
            
            # Display the final MPA report
            display(Markdown(final_mpa_report))
        else:
            print(f"Unsupported file type: {filename}")

# Handle the uploaded files and run the crew
def handle_and_run(change):
    files_content = handle_upload(change)
    run_crew_with_files(files_content)

# Attach the handler to the file upload widget
upload.observe(handle_and_run, names='value')

In [ ]:
# Try it Yourself

topic = "Financial Analysis of XYZ Corp"
result = crew.kickoff(inputs={"topic": topic})

Markdown(result)